# Demonstration of AgnosticQA methods
### Riley Thai, 27 October 2023

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib

import numpy as np

GUI event loop hook disabled.


Preamble for my system to load code directory of the repository.

In [ ]:
import sys
sys.path.insert(0, "/home/riley/uni/rproj/code")

# First look at data.

First step is reading in and cleaning the files -- this should become antequated since we want to read SQL tables instead.

In [2]:
from astropy.table import Table
from cleaners import clean_ASPCAP

t = Table.read("/home/riley/uni/rproj/data/allASPCAPVisit-0.4.0.fits").to_pandas()
data = clean_ASPCAP(t)
data

        ID SDSS_ID HEALPIX  ... RAW_E_TI_2_H RAW_V_H RAW_E_V_H
24030  NaN     NaN     NaN  ...          NaN     NaN       NaN
24031  NaN     NaN     NaN  ...          NaN     NaN       NaN
54810  NaN     NaN     NaN  ...          NaN     NaN       NaN
54811  NaN     NaN     NaN  ...          NaN     NaN       NaN
18194  NaN     NaN     NaN  ...          NaN     NaN       NaN
...    ...     ...     ...  ...          ...     ...       ...
24778  NaN     NaN     NaN  ...          NaN     NaN       NaN
71845  NaN     NaN     NaN  ...          NaN     NaN       NaN
43323  NaN     NaN     NaN  ...          NaN     NaN       NaN
43288  NaN     NaN     NaN  ...          NaN     NaN       NaN
71805  NaN     NaN     NaN  ...          NaN     NaN       NaN

[36344 rows x 331 columns]

We can see that this is just the standard ASPCAP output table, except the cleaning routine masks those with invalid paramter values or uncertainties.

We can do some very simple exploratory analysis of our dataset with one of our plotter functions.

In [ ]:
from plotters import plot_uncertainties_hist

plot_uncertainties_hist(data,show=True)

The above plot showcases a logarithmic histogram of the uncertainties of our given parameters.

The "show" option is specifically implemented for notebook workflows. It's the same as doing any plotter function with a show call.

In [ ]:
plot_uncertainties_hist(data)
plt.show(block=False)

# Visit Frequencies

As a simple introduction, let's use the example of visit frequencies to see how things are implemented. We do the following for everything.

1. Pass it through a calculation method.
2. Pass this output table through a plotting method(s).

In [ ]:
# calculation
from calculators import calculate_visit_frequency

visit_data = calculate_visit_frequency(data, fn="AllASPCAPVisit-0.4.0",write=False)

In [ ]:
# plotting
from plotters import plot_visit_frequency

plot_visit_frequency(visit_data)
plt.show(block=False)

# Uncertainty z-scores (pairwise distance comparisons)

The main thing I did for this undergrad project was pairwise distance comparisons to assess how uncertainties reflected actual measurement variations.

This is what we call the 'z-score'.

We can calculate some z-scores for effective temperature ($T_{\mathrm{eff}}$) and metallicity ($\mathrm{[Fe/H]}$)

In [3]:
from calculators import calculate_pairwise_distance

z_data = calculate_pairwise_distance(data, ["TEFF", "FE_H"], "allASPCAPVisit-0.4.0", write=False)

Time for calculation: 7.019311485000799


Data is stored in a nice table, where everything is indexed by the same key as the original.

In [4]:
z_data

            TEFF  FE_H
0      -1.337591   NaN
1            NaN   NaN
2            NaN   NaN
3            NaN   NaN
4            NaN   NaN
...          ...   ...
273687       NaN   NaN
273688       NaN   NaN
273689       NaN   NaN
273690       NaN   NaN
273691       NaN   NaN

[273692 rows x 2 columns]

We can plot basic histograms of z-scores, with some fractional adjustments

In [ ]:
from plotters import plot_pairwise

plot_pairwise(z_data, show=True)
plot_pairwise(z_data, adjust='frac',show=True)

Each of the plotters functions also saves each figure as its own PDF file. The filepath can be customized with the "figpath" option.

Too, the specific variable(s) to plot can be specified with the vars option. 

In [ ]:
plot_pairwise(z_data,vars=["TEFF"],figpath='./',show=False) 

Each plotting method is hard-coded in its output, but can be customized with options, and is adaptive to the data given to it.

 If we want to instead view it altogether, we can use the "overlap" option.

In [ ]:
plot_pairwise(z_data, overlap=True,show=True)

Next, lets recalculate our z-scores, adding in some additional systematic uncertainties, and another set with a gridsearched 3 parameter error model.

Here I'm using an option to force the lower bound on the uncertainty cleaning routine to be zero.

In [5]:
data = clean_ASPCAP(t, zero_lower_bound=True)
z_data = calculate_pairwise_distance(data, ["TEFF", "FE_H"], "allASPCAPVisit-0.4.0", write=True, gridsearch= True, adjust=True)

# we must mask any infinities from the division in the z-score calculation
z_data.replace([np.inf, -np.inf], np.nan, inplace=True)

TEFF: 0.9870594251353614 || (array([46.79252078]), array([100.]), array([100.]))
FE_H: 1.0034592653730792 || (array([0.04679252]), array([0.1]), array([0.57368421]))
Time for calculation: 38.893130237998776


Note the "write" parameter, which saves it as a parquet file. Parquet is used because it's just very good and fast (especially compared to fits tables).

In [6]:
import pandas as pd
from datetime import date
%timeit pd.read_parquet(f"data/z_{date.today()}_allASPCAPVisit-0.4.0.parquet")

25.8 ms +- 361 us per loop (mean +- std. dev. of 7 runs, 10 loops each)


Here, the parameter values are saved in a *sketchy* way. Any added systematic uncertainty has a XXX+10, where 10 is replaced with the added uncertainty. 

Gridsearch is "_GS", and the parameters are also saved as _tX, where X is the parameter number

It should be noted that if we want to save these as fits for legacy, we need a different method of storing them.

In [7]:
z_data

            TEFF  TEFF+10.0  TEFF+25.0  ...   FE_H_t0  FE_H_t1   FE_H_t2
0      -1.337591  -1.323694  -1.245670  ...  0.046793      0.1  0.573684
1      -2.905979  -2.426800  -1.427224  ...       NaN      NaN       NaN
2      -1.154266  -0.744745  -0.345436  ...       NaN      NaN       NaN
3       1.689043   1.067025   0.489291  ...       NaN      NaN       NaN
4       6.092155   4.245802   2.068219  ...       NaN      NaN       NaN
...          ...        ...        ...  ...       ...      ...       ...
435901       NaN        NaN        NaN  ...       NaN      NaN       NaN
435902       NaN        NaN        NaN  ...       NaN      NaN       NaN
435903       NaN        NaN        NaN  ...       NaN      NaN       NaN
435904       NaN        NaN        NaN  ...       NaN      NaN       NaN
435905       NaN        NaN        NaN  ...       NaN      NaN       NaN

[435906 rows x 28 columns]

In [8]:
# theta 1, for example
z_data["TEFF_t1"]

0         100.0
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
435901      NaN
435902      NaN
435903      NaN
435904      NaN
435905      NaN
Name: TEFF_t1, Length: 435906, dtype: float64

Notice all the NaN's of empty space. 

Plotting...

In [ ]:
plot_pairwise(z_data, adjust="sys",show=True)
plot_pairwise(z_data, overlap=True, adjust="grid",show=True)

To have a look at the impact of our added uncertainties, let's plot how it changes the standard dev. of the z-score distribution.

In [ ]:
from plotters import plot_added_uncertainty_sigma

plot_added_uncertainty_sigma(z_data,show=True)

We can do a surface density plot for the uncertainties of our gridsearched uncertainties to have a look at which error parameters are dominant.

In [9]:
from plotters import plot_binned_statistic

additive = z_data["FE_H_t2"].dropna().values*z_data["FE_H_TEFF"].dropna().values + z_data["FE_H_t1"].dropna().values * z_data["FE_H_SNR"].dropna().values + z_data["FE_H_t0"].dropna().values
plot_binned_statistic(z_data["FE_H_TEFF"].dropna().values,additive,xlabel=r'$T_{\mathrm{eff}}$',ylabel=r"$\sigma_+$",show=True)

<Figure size 760x940 with 2 Axes>

# Coadd Delta

Now I'm going to calculate the coadd delta -- the difference between the visit and coadd values. To do this, we first must specify the stars file in the cleaning routine.

In [10]:
from calculators import calculate_coadd_delta
from util import Lookup
from astropy.table import Table
from cleaners import clean_ASPCAP

t1 = Table.read("data/allASPCAPVisit-0.4.0.fits").to_pandas()
t2 = Table.read("data/allASPCAPStar-0.4.0.fits").to_pandas()

data_visits,data_stars = clean_ASPCAP(t1,t2)

# drop rows above SNR > 200
data_visits = data_visits.mask(data_visits["SNR"] > 200)

coadd_data = calculate_coadd_delta(data_visits, data_stars,"allASPCAP-0.4.0", write=False)
coadd_data

             TEFF   LOGG   FE_H  ...    V_H         SNR  TELESCOPE
0       55.356934  0.123  0.058  ...    NaN   33.973999   b'apo1m'
1       74.845703  0.131  0.035  ...    NaN   82.559898   b'apo1m'
2      203.166016  0.357  0.126  ...    NaN  135.419006   b'apo1m'
3      140.775879  0.352  0.046  ...    NaN   72.371902   b'apo1m'
4        9.197021  0.009  0.001  ...    NaN   25.364201  b'apo25m'
...           ...    ...    ...  ...    ...         ...        ...
18771    0.822998  0.053  0.013  ...  0.024   66.715500  b'lco25m'
18772    4.293945  0.058  0.022  ...  0.001   57.420399  b'lco25m'
18773   12.010010  0.002  0.023  ...  0.044   55.549801  b'lco25m'
18774    8.687988  0.025  0.019  ...  0.025   60.342098  b'lco25m'
18775    0.321045  0.011  0.005  ...  0.112   58.738499  b'lco25m'

[18776 rows x 27 columns]

We can do a few types of plots comparing the coadd delta to the SNR, such as these binned plots that compare median coadd delta to SNR.

In [ ]:
from plotters import plot_delta_coadd, plot_compare_coadd_snr


plot_delta_coadd(coadd_data, vars=["LOGG"],show=True)

We can also use any callable statistic array operation, such as the root mean square.

In [ ]:
from util import rms
plot_delta_coadd(coadd_data, vars=["TI_H"],type=rms,show=True)

We can also do plots grouped by nucleosynthetic processes. Here, these ones also overplot the precision targets for the survey.

In [11]:
plot_compare_coadd_snr(coadd_data,show=True)

plot_compare_coadd_snr(coadd_data,compare_telescope=True,show=True)

ALERT: O_H all NaN for APO
ALERT: SI_H all NaN for APO
ALERT: S_H all NaN for APO
ALERT: TI_H all NaN for APO
ALERT: N_H all NaN for APO


In [12]:
from datetime import date
print(date.today())
print(matplotlib.__version__)

2023-10-27
3.7.1
